In [90]:
import random as rand
import math as mt
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt


In [91]:
def Compute_Probabilities(Weight_Array):              # (1)
    Total = sum(Weight_Array)
    Choosen_Probabilities = [Weight / Total for Weight in Weight_Array]
    return Choosen_Probabilities


def Compute_Weight(rank_j, np, Sigma):                              # (2)
    Expo = -((rank_j - 1)**2) / (2 * Sigma**2 * np**2)
    Weight = (1 / (Sigma * np * mt.sqrt(2 * mt.pi))) * mt.exp(Expo)
    return Weight

def Gaussian_Distr(x_d, mu_d, Delta_d):                      # (3)
    Expo = -((x_d - mu_d)**2) / (2 * Delta_d**2)
    GaussValue = (1 / (Delta_d * mt.sqrt(2 * mt.pi))) * mt.exp(Expo)
    return GaussValue

def Compute_Delta(X_D_Val, x_j_d, Xi, np):                     # (4)
    sum_of_diffs = sum(abs(x_i_d - x_j_d) for x_i_d in X_D_Val)
    Delta_d = Xi * sum_of_diffs / (np - 1)
    return Delta_d

def Compute_Sigma(FS_max_i, FS_min_i, FS_max, FS_min, Eta):       # (5)
    Expo = -(FS_max_i - FS_min_i) / (FS_max - FS_min + Eta)
    sigma_i = 0.1 + 0.3 * mt.exp(Expo)
    return sigma_i

def Compute_Mu(x_j_d, x_seed_d, DE_Mutator):                      # (6)
    return x_j_d + DE_Mutator * (x_seed_d - x_j_d)

def SingleModal_P_i(FSE_i, FSE_max):                                    # (7)
    P_i = FSE_i / FSE_max
    return P_i

def MultiModal_P_i(FSE_i, FSE_min, FSE_max, Eta):                   # (8)
    numerator = FSE_i + abs(FSE_min) + Eta
    denominator = FSE_max + abs(FSE_min) + Eta
    P_i = numerator / denominator
    return P_i

def Compute_Metrics(NPF, NKP, NR, NSR, FE):                         # (9)
    PR = sum(NPF) / (NKP * NR)
    SR = NSR / NR
    CS = sum(FE) / NR
    return PR, SR, CS


<h4>Algorithm 1: Clustering for Crowding

In [92]:
def calDis(individual, reference_point):
    total = 0
    for i in range(len(individual)):
        total += (individual[i] - reference_point[i]) ** 2
    return mt.sqrt(total)

In [93]:

def Algorithm_1(popArray, clusSize):
    clusters = []
    indexItr = []
    minDis=0
    neatInd=-1
    while len(indexItr) + clusSize <= len(popArray):
        refPoint = [rand.random() for _ in popArray[0]]
        minDis = float('inf')
        nearInd = -1
        
        for i in range(len(popArray)):
            if i in indexItr:
                continue
            dis = calDis(popArray[i], refPoint)
            if dis < minDis:
                minDis = dis
                nearInd = i
                
        indNear = popArray[nearInd]
        indexItr.append(nearInd)
        cluster = [indNear]
        
        disArray = []
        for i in range(len(popArray)):
            if i in indexItr:
                continue
            dis = calDis(popArray[i], indNear)
            disArray.append((dis, i))
        
        disArray.sort()
        
        for j in range(clusSize - 1):
            if j < len(disArray):
                index = disArray[j][1]
                cluster.append(popArray[index])
                indexItr.append(index)
        
        elemPop = []
        for i in range(len(popArray)):
            if i not in indexItr:
                elemPop.append(popArray[i])
        popArray = elemPop
        
        clusters.append(cluster)
    
    return clusters


<h4>Algorithm 2 Clustering for Speciation

In [94]:
def combinePopAndFit(comPopFit, popArray, fitnessVal):
    for i in range(len(popArray)):
        comPopFit.append((popArray[i], fitnessVal[i]))  

def sortCombineArray(comPopFit):
    for i in range(len(comPopFit)):
        for j in range(i + 1, len(comPopFit)):
            if comPopFit[i][1] < comPopFit[j][1]:
                temp = comPopFit[i]
                comPopFit[i] = comPopFit[j]
                comPopFit[j] = temp

In [95]:
def Algorithm_2(popArray, fitnessVal, clusSize):
    specieArray = []
    comPopFit = []

    while len(popArray) >= clusSize:
        combinePopAndFit(comPopFit, popArray, fitnessVal)
        sortCombineArray(comPopFit)
        
        popArray = []
        fitnessVal = []
        for x in comPopFit:
            popArray.append(x[0])
            fitnessVal.append(x[1])

        bestInd = popArray[0]
        specieCluster = [bestInd]

        disArray = []
        for i in range(1, len(popArray)):
            dis = calDis(popArray[i], bestInd)
            disArray.append((dis, i))

        disArray.sort()

        for j in range(clusSize - 1):
            if j < len(disArray):
                index = disArray[j][1]
                specieCluster.append(popArray[index])

        elemPop = []
        elemFit = []
        for i in range(len(popArray)):
            if popArray[i] not in specieCluster:
                elemPop.append(popArray[i])
                elemFit.append(fitnessVal[i])

        popArray = elemPop
        fitnessVal = elemFit

        specieArray.append(specieCluster)

    return specieArray

<h4>Algorithm 4 Solution Construction for Ants

In [96]:
def Algorithm_4(NS, Niches, FS_max, FS_min, Eta, F, Xi, DE_Mutator):
    Fitted_Solutions = []

    Idx = 0
    while Idx < len(Niches):  # Step 1
        LNiche = Niches[Idx]
        FS_max_i = LNiche['FS_max_i']  # 1.1
        FS_min_i = LNiche['FS_min_i']  # 1.1
        Niche_Solutions = LNiche['solutions']

        Sigma = Compute_Sigma(FS_max_i, FS_min_i, FS_max, FS_min, Eta)  # 1.2

        Weight_Array = []
        Rank = 0
        while Rank < len(Niche_Solutions):
            Weight = Compute_Weight(Rank + 1, len(Niche_Solutions), Sigma)
            Weight_Array.append(Weight)  # 1.3
            Rank += 1

        Choosen_Probabilities = Compute_Probabilities(Weight_Array)  # 1.3

        ns_count = 0
        while ns_count < NS:  # 1.4
            Idx_Chosen = rand.choices( population=list(range(len(Niche_Solutions))), weights=Choosen_Probabilities, k=1 )[0]
            Chosen_Sol = Niche_Solutions[Idx_Chosen]

            if rand.random() <= 0.5:  # 1.4.2
                Updated_Sol = Chosen_Sol[:]
            else:
                Seed_Count = [i for i in range(len(Niche_Solutions)) if i != Idx_Chosen]
                Seed_Idx = rand.choice(Seed_Count)
                Seed_Sol = Niche_Solutions[Seed_Idx]
                Updated_Sol = []
                j = 0
                while j < len(Chosen_Sol):
                    mu_d = Compute_Mu(Chosen_Sol[j], Seed_Sol[j], DE_Mutator)
                    Updated_Sol.append(mu_d)
                    j += 1

            Solution = []
            j = 0
            while j < len(Updated_Sol):
                X_D_Val = [Sol[j] for Sol in Niche_Solutions]
                Delta_d = Compute_Delta(X_D_Val, Chosen_Sol[j], Xi, NS)  # 1.4.3
                GaussValue = Gaussian_Distr(Chosen_Sol[j], Updated_Sol[j], Delta_d)  # 1.4.4
                Solution.append(GaussValue)
                j += 1

            Fitness = F(Solution[0])
            Fitted_Solutions.append((Solution, Fitness))
            ns_count += 1

        Idx += 1

    return Fitted_Solutions


<h4>Algorithm 5: Adaptive Local Search

In [97]:
def Algorithm_5(S, FSE, Delta, N, Eta):
    FSE_min = min(FSE)                  # Step 1
    FSE_max = max(FSE)                  # Step 1
    Flag = False                        # Step 1

    if FSE_min <= 0:                                # Step 2
        FSE_max = FSE_max + abs(FSE_min) + Eta      # Step 2
        Flag = True                                 # Step 2

    Prob_Array = []
    for Fitness in FSE:                             # Step 3
        if Flag:                                #IF MultiModal
            prob = MultiModal_P_i(Fitness, FSE_min, FSE_max, Eta)
        else:                                   #IF SingleModal
            prob = SingleModal_P_i(Fitness, FSE_max)
        Prob_Array.append(prob)

    for i in range(len(S)):                             # Step 4
        if rand.random() <= Prob_Array[i]:
            for _ in range(N):
                New_Val = [
                    rand.gauss(S[i][j], Delta) for j in range(len(S[i]))
                ]
                if sum(New_Val) > sum(S[i]):
                    S[i] = New_Val
                    FSE[i] = sum(S[i])

    return S, FSE

<h4>Algorithm 6: Local Search-Based AMC-ACO (LAMC-ACO)


In [98]:
def Algorithm_6(NP, G, Delta, Eta, F, Xi, Iteration_Count, DE_Mutator=0.5):
    LAMC_Sol = []
    i = 0
    while i < NP:
        Sol = [rand.random() for _ in range(10)]
        Fitness = F(Sol[0])
        LAMC_Sol.append((Sol, Fitness))
        i += 1

    FS_Max = max(Fitness for _, Fitness in LAMC_Sol)
    FS_Min = min(Fitness for _, Fitness in LAMC_Sol)

    NS = rand.choice(G)

    Niches = Algorithm_1([Sol for Sol, _ in LAMC_Sol], NS)

    Processed_Niches = []
    Niche_Idx = 0
    while Niche_Idx < len(Niches):
        LNiche = Niches[Niche_Idx]
        FS_Max_I = max(F(Sol[0]) for Sol in LNiche)
        FS_Min_I = min(F(Sol[0]) for Sol in LNiche)
        Processed_Niches.append({
            'FS_max_i': FS_Max_I,
            'FS_min_i': FS_Min_I,
            'solutions': LNiche
        })

        Niche_Idx += 1

    Fitted_Solutions = Algorithm_4( NS, Processed_Niches, FS_Max, FS_Min, Eta, F, Xi, DE_Mutator )

    i = 0
    while i < len(Fitted_Solutions): 
        Updated_Sol, Fitness = Fitted_Solutions[i]
        Nearest_Solution, Closest_Fitting = min( LAMC_Sol, key=lambda X: sum(abs(A - B) for A, B in zip(X[0], Updated_Sol))
        )
        if Fitness > Closest_Fitting:
            LAMC_Sol.remove((Nearest_Solution, Closest_Fitting))
            LAMC_Sol.append((Updated_Sol, Fitness))
        i += 1

    LAMC_Sol_Sol, LAMC_Sol_Fitness = zip(*LAMC_Sol)
    LAMC_Sol_Sol, LAMC_Sol_Fitness = Algorithm_5(
        list(LAMC_Sol_Sol), list(LAMC_Sol_Fitness), Delta, 1, Eta
    )

    Iteration_Count -= 1
    if Iteration_Count > 0:
        return Algorithm_6( NP, G, Delta, Eta, F, Xi, Iteration_Count, DE_Mutator )
    else:
        return LAMC_Sol


<h4>Algorithm 7: Local Search-Based AMS-ACO (LAMS-ACO)



In [99]:
def Algorithm_7(NP, G, Delta, Eta, F, Xi, Max_Generations):
    LAMS = []
    i = 0
    while i < NP:  
        Sol = [rand.random() for _ in range(10)]
        Fitness = F(Sol[0])  # Using Function_One for Fitness calculation
        LAMS.append((Sol, Fitness))
        i += 1

    FS_Max = max(Fitness for _, Fitness in LAMS)
    FS_Min = min(Fitness for _, Fitness in LAMS)

    Clus_Size = len(LAMS) // 5 if len(LAMS) >= 5 else len(LAMS)
    NS = rand.choice(G)

    Species = Algorithm_2([Sol for Sol, _ in LAMS], [Fit for _, Fit in LAMS], Clus_Size)

    Fitted_Solutions = Algorithm_4(NS, Species, FS_Max, FS_Min, Eta, F, Xi)

    LNiche_Index = 0
    while LNiche_Index < len(Species):
        LNiche = Species[LNiche_Index]
        LAMS_Sol = LNiche["solutions"]
        
        j = 0
        while j < len(Fitted_Solutions):
            Updated_Sol, Fitness = Fitted_Solutions[j]
            Nearest_Solution, Closest_Fitting = min([(Sol, sum(Sol)) for Sol in LAMS_Sol], key=lambda X: sum([abs(A - B) for A, B in zip(X[0], Updated_Sol)]))
            if Fitness > Closest_Fitting:
                LAMS_Sol.remove(Nearest_Solution)
                LAMS_Sol.append(Updated_Sol)
            j += 1
        
        LNiche_Index += 1

    LAMS_Sol, LAMS_Fitness = zip(*LAMS)
    LAMS_Sol, LAMS_Fitness = Algorithm_5(list(LAMS_Sol), list(LAMS_Fitness), Delta, 1, Eta)

    return list(zip(LAMS_Sol, LAMS_Fitness))


In [100]:
def Function_One(x):
    if 0 <=x and x<2.5:
        return 80 * (2.5 - x)
    elif 2.5 <=x and x<5.0:
        return 64 * (x - 2.5)
    elif 5.0 <=x and  x<7.5:
        return 64 * (7.5 - x)
    elif 7.5 <=x and x<12.5:
        return 28 * (x - 7.5)
    elif 12.5 <=x and x<17.5:
        return 28 * (17.5 - x)
    elif 17.5 <=x and x< 22.5:
        return 32 * (x - 17.5)
    elif 22.5 <=x and x <27.5:
        return 32 * (27.5 - x)
    elif 27.5 <=x and x<=30:
        return 80 * (x - 27.5)
    else:
        return 0

In [101]:
def Function_Two(x):
    Fun=np.sin(5*np.pi*x)**6
    return Fun

In [102]:
def Function_Three(x):
    Fun=np.exp(-2*np.log(2)*((x-0.08)/(0.854))**2)* (np.sin(5*np.pi*(x**(3/4)-0.05))**6)
    return Fun

In [103]:
def Function_Four(x,y):
    Fun=(x**2 + y - 11)**2 + (x + y**2 - 7)**2
    return Fun

In [104]:
def Function_Five(x, y):
    Fun=(4-2.1*x**2+(x**4)/3)*x**2+x*y+(4*y**2-4)*y**2
    return Fun

In [105]:
def Function_Six(x):
    size=len(x)
    sum=0
    for i in range(size):
        prod=1
        for j in range(1,6):
            prod=prod*(j*np.cos((j+1)*x[i]+j))
        sum=sum+prod
    return -sum

In [106]:
def Function_Seven(x):
    size = len(x)
    sum=0
    for i in range (size):
        sum+=np.sin(10*np.log(x[i]))
                    
    result =sum/size
    return result

In [107]:
def Function_Eight(x,k):
    size=len(x)
    sum=0
    for i in range(size):
        sum=sum+ (10+9*np.cos(2*np.pi*k[i]*x[i]))
    
    funVal=sum*-1
    return funVal
    

<H4>F1 using Algorithm 6

In [108]:
NP = 20
G = [2, 5, 10]
Delta = 0.1
Eta = 1e-4
Xi = 0.5
Iterations = 50

def Fitness_Fx(x):
    return Function_One(x)

archive = Algorithm_6(NP, G, Delta, Eta, Fitness_Fx, Xi, Iterations)

print("Final archive solutions and Fitness:")
for Sol, Fitness in archive:
    print(f"solutions: {Sol}, {Fitness}")


Final archive solutions and Fitness:
solutions: [0.5366908036964605, 0.3154339097812462, 0.1369278853512873, 0.10219420748710839, 0.8616052616393656, 0.314049530440068, 0.015106731148271102, 0.015434403071865388, 0.9556342204336101, 0.112721748596472], 157.06473570428315
solutions: [0.577685442571157, 0.19301331147480294, 0.14398650956882186, 0.21437264543327905, 0.0514078324202486, 0.5042006243173298, 0.33386315025501867, 0.7703805643586932, 0.3213795116616631, 0.24127983217547488], 153.78516459430745
solutions: [0.012434306054178612, 0.13628756972320544, 0.21337063150591162, 0.5681988830153805, 0.8879949304323261, 0.2966110788221449, 0.5650136640731223, 0.07721573591018682, 0.8671199438090608, 0.4896190290232705], 199.0052555156657
solutions: [0.056743951230015965, 0.6930518763595351, 0.8538575130404166, 0.9310593873772575, 0.9668355895601257, 0.9647776903877634, 0.8047101537875158, 0.1586842573333589, 0.29809761364945675, 0.3556762230460556], 195.4604839015987
solutions: [0.10778098

<H4>F2 using Algorithm 6

In [109]:

NP = 20
G = [2, 5, 10]
Delta = 0.1
Eta = 1e-4
Xi = 0.5
Iterations = 50

def Fitness_Fx(x):
    return Function_Two(x)

archive = Algorithm_6(NP, G, Delta, Eta, Fitness_Fx, Xi, Iterations)

print("Final archive solutions and Fitness:")
for Sol, Fitness in archive:
    print(f": {Sol}, {Fitness}")


Final archive solutions and Fitness:
: [0.30273891509442274, 0.6570344288994925, 0.1109412544198779, 0.043757279229707624, 0.9853363173669986, 0.5472099730022854, 0.34843801634007654, 0.9354850024985784, 0.6127114598689613, 0.4285353369224226], 0.9944608061493658
: [0.16734946597041833, 0.7263518545652126, 0.21322162465972416, 0.8277813253482188, 0.7894215489388372, 0.7290774002845474, 0.16534292223748115, 0.3391014861720064, 0.8241306563639921, 0.43364191918302153], 0.013957443342863023
: [0.9689441480905147, 0.6408979356310816, 0.3460826473269988, 0.3490139765559499, 0.8557567398011461, 0.7702667320042836, 0.4803730925580475, 0.4768079989523274, 0.3314397109397106, 0.5405446932899179], 0.0106022344259439
: [0.7971790704486399, 0.5342764133502524, 0.3992302791761002, 0.4832933666327125, 0.13307027649627445, 0.8293742851690867, 0.8923798440569521, 0.6965254450304718, 0.39811924661804565, 0.803685089751206], 7.554746997293287e-09
: [0.6561348619114503, 0.3405502211078367, 0.172163106566